# 第4章: 形態素解析

夏目漱石の小説『吾輩は猫である』の文章（neko.txt）をMeCabを使って形態素解析し，その結果をneko.txt.mecabというファイルに保存せよ．このファイルを用いて，以下の問に対応するプログラムを実装せよ．

なお，問題37, 38, 39はmatplotlibもしくはGnuplotを用いるとよい．

In [ ]:
import MeCab
tagger = MeCab.Tagger()
with open("./data/neko.txt","r") as f:
    mecab_result = tagger.parse(f.read())
with open("./data/neko.txt.mecab","w") as f:
    f.write(mecab_result)

## 30. 形態素解析結果の読み込み
### 形態素解析結果（neko.txt.mecab）を読み込むプログラムを実装せよ．ただし，各形態素は表層形（surface），基本形（base），品詞（pos），品詞細分類1（pos1）をキーとするマッピング型に格納し，1文を形態素（マッピング型）のリストとして表現せよ．第4章の残りの問題では，ここで作ったプログラムを活用せよ．

In [ ]:
def read_mecab_file(file_name:str):
    with open(file_name,"r") as f:
        f.readline()
        f.readline()
        result = []
        for row in f.readlines():
            if row == 'EOS\n':
                break
            text = row.split("\t")
            pos = text[1].replace("\n","").split(",")
            dic = {"surface" : text[0],
                   "base" : pos[6],
                   "pos" : pos[0],
                   "pos1" : pos[1]}
            result.append(dic)
    return result
mecab_file = read_mecab_file("./data/neko.txt.mecab")
mecab_file

## 31. 動詞Permalink
### 動詞の表層形をすべて抽出せよ．

In [ ]:
import pandas as pd
data = pd.DataFrame(mecab_file)
data.surface.unique()

## 32. 動詞の基本形Permalink
### 動詞の基本形をすべて抽出せよ．

In [ ]:
data.base.unique()

## 33. 「AのB」Permalink
### 2つの名詞が「の」で連結されている名詞句を抽出せよ．

In [ ]:
def mk_words(data:pd.DataFrame,index_list:list):
    return "".join([data.loc[index].surface for index in index_list])

result = []
for index in data[data["surface"] == "の"].index:
    before = data.loc[index-1]
    after = data.loc[index+1]
    if before["pos"] == "名詞" and after["pos"] == "名詞":
        result.append(mk_words(data,[index-1,index,index+1]))
result

## 34. 名詞の連接Permalink
### 名詞の連接（連続して出現する名詞）を最長一致で抽出せよ．

In [ ]:
def continue_index(index_list:list):
    start_index = 0
    continue_lists = []
    continue_list = []
    for index in index_list:
        if start_index != index:
            if len(continue_list) > 1:
                continue_lists.append(continue_list)
            continue_list = []
            start_index = index
        continue_list.append(index)
        start_index += 1
    return continue_lists

continue_noun_list = continue_index(data[data.pos == "名詞"].index)
connected_noun = [mk_words(data,noun_list) for noun_list in continue_noun_list]
connected_noun

## 35. 単語の出現頻度Permalink
### 文章中に出現する単語とその出現頻度を求め，出現頻度の高い順に並べよ．

In [ ]:
data.groupby("surface").count().sort_values("base",ascending=False)

## 36. 頻度上位10語Permalink
### 出現頻度が高い10語とその出現頻度をグラフ（例えば棒グラフなど）で表示せよ．

In [ ]:
import matplotlib.pyplot as plt
import japanize_matplotlib

data[data.pos.isin(["名詞"])].groupby("surface").base.count().rename("count").sort_values(ascending=False).head(10).plot.bar()

## 37. 「猫」と共起頻度の高い上位10語Permalink
### 「猫」とよく共起する（共起頻度が高い）10語とその出現頻度をグラフ（例えば棒グラフなど）で表示せよ．

In [ ]:
start_index = 0
result_df = pd.DataFrame()
for period_index in data[data.pos1 == "句点"].index:
    neko_df = data.loc[start_index:period_index]
    if not neko_df[neko_df.surface == "猫"].empty:
        result_df = pd.concat([result_df,neko_df])
    start_index = period_index + 1
result_df = result_df[result_df.surface != "猫"]
result_df[result_df.pos.isin(["名詞"])].groupby("surface").base.count().rename("count").sort_values(ascending=False).head(10).plot.bar()

## 38. ヒストグラムPermalink
### 単語の出現頻度のヒストグラムを描け．ただし，横軸は出現頻度を表し，1から単語の出現頻度の最大値までの線形目盛とする．縦軸はx軸で示される出現頻度となった単語の異なり数（種類数）である．

In [ ]:
data.groupby("surface").base.count().rename("count").reset_index().groupby("count").count().reset_index().plot.scatter(y="surface",x="count")

## 39. Zipfの法則Permalink
### 単語の出現頻度順位を横軸，その出現頻度を縦軸として，両対数グラフをプロットせよ．

In [ ]:
data.groupby("surface").base.count().rename("count").reset_index().groupby("count").count().reset_index().plot.scatter(y="surface",x="count",logx=True,logy=True)